In [1]:
import sys
sys.path.append('/home/ranvir/AiCore')
from database_connection import DatabaseConnection
db = DatabaseConnection()

<h1 style="color: rgb(241, 90, 36)"><img src="https://github.com/AI-Core/Content-Public/blob/main/Content/units/Data-Handling/3.%20SQL/11.%20SQL%20Common%20Aggregations/images/SQLIcon.png?modified=2233&raw=1" width=80px height=80px style="vertical-align: middle;"> SQL Common Aggregations</h1>

When working in the domain of data, once the data is stored in the database, you might need to gather insight from it. What use is there in gathering data if we can't apply some analysis to draw key insights from it? This can be done with the use of SQL *aggregation functions*. **Aggregation functions** perform calculations on a set of values to return a single value.

Here are the most commonly used **aggregation functions**:

- <b style="color: rgb(241, 90, 36)">COUNT</b> : To count how many rows are in a particular column
- <b style="color: rgb(241, 90, 36)">SUM</b> : To `SUM` all rows in a particular column
- <b style="color: rgb(241, 90, 36)">MIN/MAX</b> : To get the **maximum** or **minimum** values in a column
- <b style="color: rgb(241, 90, 36)">AVG</b> : To return the **average** value in a column

These are the most commonly used aggregation function in PostgreSQL, though there are many more. If you would like to find out what's possible all other functions can be seen at this [link.](https://www.postgresql.org/docs/9.5/functions-aggregate.html)

## `COUNT`

`COUNT` allows you to count the number of **non-NULL** values in a column, the syntax is:


In [ ]:
COUNT({column name});

We can apply the `COUNT` to any column in our `SELECT` statement:

In [2]:
db.execute_query('''
                    SELECT COUNT(address)
                    FROM address;
                ''')

,count
0,603


Which returns a value of `603`. Remember that `COUNT` doesn't count `NULL` values so applying it to the `address2` column we get a different answer:

In [4]:
db.execute_query('''
                    SELECT COUNT(address2)
                    FROM address;
                ''')

,count
0,599


Which returns a value of `599`. There are `603` records in the `address` table so we know that the `address2` column has four `NULL` values in it.

`total rows - COUNT(address2)` = `4`

## `SUM`

`SUM`, sums all the values in a column, it only works on `numerical` data (we can't sum the `address` column for example). If there is a `NULL` value in the column it is ignored. The syntax is:

In [ ]:
SUM({column name})

In [5]:
db.execute_query('''
                    SELECT SUM(replacement_cost)
                    FROM film;
                ''')

,sum
0,19984.0


Looks like the replacement value of all stock is 20k interesting to know for a business.

## `MIN/MAX`

`MIN` returns the **minimum value** in a column and `MAX` returns the **maximum value**. They can be applied to **numerical data**, **array** or **character** columns. Both functions have the syntax:

In [ ]:
MIN({column name})
MAX({column name})

Let's get the minimum `replacement_cost` and maximum `replacement_cost` of a film in the `film` table:

In [6]:
db.execute_query('''
                    SELECT MIN(replacement_cost) AS minimum_replacement_cost,
                        MAX(replacement_cost) AS maximum_replacement_cost
                    FROM
                        film;
                ''')

,minimum_replacement_cost,maximum_replacement_cost
0,9.99,29.99


>You'll often want to alias your aggregation columns, since by default, the aggregated column will be named by the name of the aggregation. So the intended result of the aggregation won't be clear. For example `SUM(replacement_cost)` creates a column called `sum` .

## `AVG`

`AVG` returns the average value in a column. It ignores `NULL`'s in both the *numerator* and *denominator* and only works on numerical values:

In [ ]:
AVG({column name})

Getting the `average_replacement` cost of a film:

In [9]:
db.execute_query('''
                    SELECT AVG(replacement_cost) AS average_replacement_cost
                    FROM film;
                ''')

,average_replacement_cost
0,19.984


<h2 style="color: rgb(241, 90, 36)">Calculations with aggregations</h2>

You can also perform *arithmetic operations* with the results of an aggregation to achieve complex analysis of data. This can be done with the standard operators `+ - / * %`.

Calculating the range of cost between the **maximum** and **minimum** `replacement_cost`:

In [11]:
db.execute_query('''
                    SELECT (MAX(replacement_cost) - MIN(replacement_cost)) AS replacement_cost_range
                    FROM film;
                ''')

,replacement_cost_range
0,20.0


`MAX` and `MIN` can be performed on dates so we could calculate how long the business has been renting out movies for:

In [13]:
db.execute_query('''
                    SELECT MAX(payment_date)- MIN(payment_date) AS payment_span
                    FROM payment;
                ''')

,payment_span
0,109 days 16:22:33


Notice this returns an interval since we're getting the time span between two dates.

Let's calculate the average `replacement_cost` as a percentage of total stock `replacement_cost`:

In [17]:
# Double percentage for sql_alchemy
db.execute_query('''
                    SELECT (AVG(replacement_cost) / SUM(replacement_cost)) * 100 AS "average_replacement_cost(%%)"
                    FROM film;
                ''')

,average_replacement_cost(%)
0,0.1


## `DISTINCT`

By default SQL will return all rows even duplicate rows. Sometimes you may want to return only the rows where column values are unique. This can be done by applying the keyword `DISTINCT` to a column. The syntax of `DISTINCT` is:


In [ ]:
DISTINCT {column1, column2, column3 ....}

Any column specified after the `DISTINCT` keyword will return only unique values from that column. This can be a great way to check a column to understand all the values the column contains. Imagine we want to check what the different types of ratings a movie could have, we could run the query:

In [18]:
db.execute_query('''
                    SELECT DISTINCT rating
                    FROM film;
                ''')

,rating
0,PG-13
1,NC-17
2,G
3,PG
4,R


Returning all the possible ratings a film could have. If we wanted to check for each of those ratings what are all the possible `rental_rate`'s, we could add the `rental_rate` column to your distinct query. Returning distinct values which are a combination of `rating` and `rental_rate`:

In [25]:
db.execute_query('''
                    SELECT 
                        DISTINCT rating,
                        rental_rate
                    FROM 
                        film
                    ORDER BY
                        rating;
                ''')

,rating,rental_rate
0,G,0.99
1,G,4.99
2,G,2.99
3,PG,2.99
4,PG,4.99
5,PG,0.99
6,PG-13,4.99
7,PG-13,2.99
8,PG-13,0.99
9,R,0.99


This can be a great way to quickly summarise the data in columns when performing initial data analysis. `DISTINCT` can also be used in conjugation with aggregations like `COUNT`. Which would return a count of unique values in a column:


In [26]:

db.execute_query('''
                    SELECT
                        COUNT(DISTINCT(rating)),
                        COUNT(DISTINCT(rental_rate))
                    FROM
                        film;
                ''')

,count,count
0,5,3


Notice here, we apply `COUNT` individually to each column  so we need to apply `DISTINCT` separately to each column.

## `DATE TRUNC`

`DATE TRUNC` stands for *date truncate* and can be a very useful function if you want to strip away values in a `timestamp`. This can help when you want to aggregate times together. The syntax is:

In [ ]:
DATE_TRUNC({field}, {column})

With `DATE_TRUNC` you can remove values from the `timestamp` up to the specified `field` value. The options for the `field` value are:

- `microseconds`
- `milliseconds`
- `second`
- `minute`
- `hour`
- `day`
- `week`
- `month`
- `quarter`
- `year`
- `decade`
- `century`
- `millennium`

If we specified `DATE_TRUNC('day', {column})` the function would truncate(remove) all values up to the date value in a timestamp.

Imagine you want to group together all the days a rental was made by `rental_date`. In the `rental` table the `rental_date` has values such as, `2005-05-24 23:03:39` and `2005-05-24 22:54:33`. Though rentals occurred on the same day, if we try to group the data by these values they won't belong to the same group since the times are different.

We can instead truncate the date up to the day value so that the value of these dates will be the same:



In [30]:
db.execute_query('''
                    SELECT DATE_TRUNC('day', rental_date)
                    FROM rental;
                ''')

,date_trunc
0,2005-05-24
1,2005-05-24
2,2005-05-24
3,2005-05-24
4,2005-05-24
...,...
16039,2005-08-23
16040,2005-08-23
16041,2005-08-23
16042,2005-08-23


After running this query you can see the values `2005-05-24 23:03:39` and `2005-05-24 22:54:33` have both been stripped of their time values, so both have the value `2005-05-24 00:00:00`. This could be useful when you want to group the entries together by day of the month. You will learn how to then group the data using a `GROUP BY` in the next lesson.

In [31]:
db.execute_query('''
                    SELECT DATE_TRUNC('day', rental_date) AS rental_day,
                        COUNT(*) AS total_daily_rentals
                    FROM rental
                    GROUP BY DATE_TRUNC('day', rental_date)
                    ORDER BY total_daily_rentals DESC;
                ''')

,rental_day,total_daily_rentals
0,2005-07-31,679
1,2005-08-01,671
2,2005-08-21,659
3,2005-07-27,649
4,2005-08-02,643
5,2005-07-29,641
6,2005-07-30,634
7,2005-08-19,628
8,2005-08-22,626
9,2005-08-20,624


## `DATE PART/EXTRACT`

The `DATE_PART` or `EXTRACT` allows us to extract a specific field from a `timestamp` or `interval`, with the following syntax:

In [ ]:
DATE_PART({field}, {timestamp/interval})
EXTRACT({field from timestamp/interval})

There is a slight difference between `EXTRACT` and `DATE_PART` though in most cases they can be used interchangeably. Since PostgreSQL 14 `EXTRACT` returns a `numeric` type and `DATE_PART` returns a `double precision` type. Double precision has precision up to 15 digits after the decimal point and numeric is up to 16383 so they can round slightly differently in rare cases.

The `field` value allows you to specify which part of the `interval` or `timestamp` you would like to extract and has the same options as `DATE_TRUNC` plus additionally:

- `timezone`: extract the timezone
- `timezone_hour`: hour component of the time zone
- `timezone_minute`: minute component of the time zone
- `dow`: numerical value for the day of the week
- `doy`: numerical value for the day of the year
- `epoch`: number of seconds past since 1970-01-01 00:00:00

An example of extracting the year from the `payment_date` in the `payment` table:



In [32]:
db.execute_query('''
                    SELECT EXTRACT('year' from payment_date) as day_taken_payment
                    FROM payment;
                ''')

,day_taken_payment
0,2007.0
1,2007.0
2,2007.0
3,2007.0
4,2007.0
...,...
16044,2007.0
16045,2007.0
16046,2007.0
16047,2007.0


What it normally looks like, for reference:

In [33]:
db.execute_query('''
                    SELECT payment_date as day_taken_payment
                    FROM payment;
                ''')

,day_taken_payment
0,2007-01-24 21:40:19.996577
1,2007-01-25 15:16:50.996577
2,2007-01-28 21:44:14.996577
3,2007-01-29 00:58:02.996577
4,2007-01-29 08:10:06.996577
...,...
16044,2007-05-14 13:44:29.996577
16045,2007-05-14 13:44:29.996577
16046,2007-05-14 13:44:29.996577
16047,2007-05-14 13:44:29.996577


## `CASE`

The `CASE` statement checks multiple conditional statements and returns a value when one is met. It evaluates each statement in order, and once one is met it will return the specified value from that statement. You will always find the `CASE` statement as part of the `SELECT` clause.

The `CASE` statement is created using the keywords `WHEN`, `THEN`, `END` and optionally `ELSE` or `AS`. Let's take a look at the syntax of the statement:

In [ ]:
CASE
    WHEN condition THEN result
    [WHEN ...]
    [ELSE result]
END AS {column alias}

First the `CASE` statement is created with the `CASE` keyword and ends when it reaches the `END` keyword. This can be followed by `AS`, to alias the newly created column from the `CASE` statement. Each condition in the `CASE` statement starts with `WHEN condition`. If this condition is met then its associated `THEN result` updates the new column with `result`.

You can think of `CASE` like `if else` statements in another programming language. The Pythonic way to view this is:

In [ ]:
if condition_1:
    return value_1
elif condition_2:
    return value_2
else:
    return value_3

Let's create a `CASE` statement to categorise each film in the `film` table based on their `rental_rate`:

In [35]:
db.execute_query('''
                    SELECT title,
                           release_year,
                           rental_rate,
                    CASE
                        WHEN rental_rate > 0 AND rental_rate < 2.99 THEN 'discount'
                        WHEN rental_rate >= 2.99 AND rental_rate < 4.99 THEN 'regular'
                        ELSE 'premium'
                    END AS quality
                    FROM
                        film;
                ''')

,title,release_year,rental_rate,quality
0,ACADEMY DINOSAUR,2006,0.99,discount
1,ACE GOLDFINGER,2006,4.99,premium
2,ADAPTATION HOLES,2006,2.99,regular
3,AFFAIR PREJUDICE,2006,2.99,regular
4,AFRICAN EGG,2006,2.99,regular
...,...,...,...,...
995,YOUTH KICK,2006,0.99,discount
996,ZHIVAGO CORE,2006,0.99,discount
997,ZOOLANDER FICTION,2006,2.99,regular
998,ZORRO ARK,2006,4.99,premium


Notice we're using the `rental_rate` column to determine which quality category each film should belong to.

We have the ranges:

- `WHEN rental_rate > 0 AND rental_rate < 2.99 THEN 'discount'`
- `WHEN rental_rate >= 2.99 AND rental_rate < 4.99 THEN 'regular'`
- `ELSE premium`

After the `CASE` statement is completed with `END`, we've aliased the column to create `AS quality`. So the first statement is saying if the `rental_rate` is between `0` and less than `2.99` then the value `discount` will be inserted into the new column `quality`. If it's between `2.99` and `4.99` then the value `regular` will be inserted. In all other cases the value `premium` will inserted into the column.

## Key Takeaways

- **Aggregations** are a powerful way to summarise your data
- **Aggregations** can be used in conjunction with the **arithmetic operators** to perform calculation on your aggregated data
- The `DISTINCT` keyword can be a great way to understand data on initial viewing. You can use it to view all unique values in a column
- `DATE PART/EXTRACT` can give retrieve almost any part of a timestamp so use it when you need time values
- The `DATE TRUNC` function can used great when you need to aggregate the data by a specific part of a **timestamp**
- The `CASE` statement can be a great way to generate new columns based on the value of other columns in your table